# Tutorial on using Propnet

The following is a tutorial designed to give a base overview of the classes and constructs used in the Propnet project. For each class an example of its construction and base usage is provided.

# Defining a Property Network: Propnet

A Graph object tells us about all the property types and models currently available for use.

The mappings contained in this object define an interconnected network of materials properties. In this form, the Graph object can be used to enumerate and analyze links between differnet materials properties.

In [1]:
from propnet.core.graph import Graph

Propnet is not intended for public use at this time. Functionality might change.



In [2]:
g = Graph()

You can print Propnet to see the property types and models it supports.

In [3]:
print(g)

Propnet Printout

Properties
	formula
	dimensionality
	structure_oxi
	oxide_type
	is_metallic
	magnetic_order
	external_identifier_mp
	computed_entry
	structure
	potentially_ferroelectric
	prototype
	composition
	null_symbol
	atomic_density
	elastic_tensor_voigt
	interplanar_spacing
	sound_velocity_longitudinal
	goldschmidt_tolerance_factor
	thermal_conductivity
	refractive_index
	ionic_radius_b
	absorption_coefficient
	p_wave_modulus
	cost_per_kg
	extinction_coefficient
	electronic_thermal_conductivity
	transmittance
	interatomic_spacing
	band_gap_pbe
	electrical_resistivity
	relative_permittivity
	snyder_acoustic_sound_velocity
	cost_per_mol
	sound_velocity_mean
	volume_unit_cell
	youngs_modulus
	dielectric_figure_of_merit_energy
	poisson_ratio
	absorbance
	specific_heat_capacity_constant_volume
	lame_first_parameter
	compliance_tensor_voigt
	hhi_reserve
	snyder_optical_sound_velocity
	reflectance
	formation_energy_per_atom
	strengthening_coefficient
	band_gap
	peierls_stress
	shear_

# Defining a Material Property

A Symbol object is used to represent types of properties (such as Young's Modulus) or conditions (such as Temperature).
- All Symbol objects are accessible in a global DEFAULT_SYMBOLS variable.
- Various metadata for each SymbolType can be accessed as shown below.

In [13]:
from propnet.symbols import DEFAULT_SYMBOLS
symbol_object = DEFAULT_SYMBOLS['youngs_modulus']
print(symbol_object)

youngs_modulus:
	name:	youngs_modulus
	category:	property
	units:	1.0 gigapascal
	object_type:	None
	display_names:	["Young's modulus", 'Elastic modulus']
	display_symbols:	['E']
	shape:	1
	comment:	



A Quantity object is used to represent values of properties (such as Young's Modulus = 200GPa) or conditions (such as temperature = 300K).

- All Quantity objects have a Symbol giving the type of property represented by the value.
- All Quantity objects must be created at runtime by specifying a value during instantiation.
- All Quantity objects have a list of strings called "tags" used to further label the property.

In [14]:
from propnet.core.quantity import Quantity
steel_youngs_modulus = Quantity('youngs_modulus', 200, ['mild steel'])
print(steel_youngs_modulus)

<youngs_modulus, 200 gigapascal, None>


# Defining a Material

A Material object is used to represent a collection of information known about a given material.

When it is first created it has no information; however, properties can be added to the material one-by-one.

In [2]:
from propnet.core.materials import Material
from propnet.core.quantity import Quantity
mild_steel = Material()
youngs_modulus = Quantity('youngs_modulus', 200, [])
mild_steel.add_quantity(youngs_modulus)
print(mild_steel)

Material: 533398e9-545c-4a5a-95be-c09aa055780b
	youngs_modulus
		<youngs_modulus, 200 gigapascal, None>



# Combining Models, Materials, and Symbols

As illustrated, a Graph object contains information for connecting many different models and symbol types. This forms an abstract web of interconnected variables without any values specified.

On the other hand, a Material object represents a grouping of values for different variables. These are represented as a collection of Quantity objects identified with the material.

At runtime, a single Graph object can be combined with one or more Material objects. This procedure allows values to be plugged in to variables. Assuming the required inputs for a model all have values, the Graph object can then dynamically predict the values for the output variables of the model.

In [1]:
from propnet.core.materials import Material
from propnet.core.graph import Graph
from propnet.core.quantity import Quantity

g = Graph()

silica = Material()
refractive_index = Quantity('refractive_index', 1.458, [])
relative_permittivity = Quantity('relative_permittivity', 3.9, [])

silica.add_quantity(refractive_index)
silica.add_quantity(relative_permittivity)

g.add_material(silica)
print(silica)

Propnet is not intended for public use at this time. Functionality might change.

Material: cc798424-97c3-4830-92a6-25c942ad01fe
	refractive_index
		<refractive_index, 1.458 dimensionless, None>

	relative_permittivity
		<relative_permittivity, 3.9 dimensionless, None>



Propnet can now examine the input values and identify if any models can be used to derive additional properties.

In this example, we've provided the relative permittivity and index of refraction of silica. Thus, using the canonical relationship from electromagnetism, we expect Propnet to properly derive the relative permeability.

Re-examining the material object previously created, a new Quantity object, the relative permeability, is now associated with that material.

In [2]:
g.evaluate(material=silica)
print(silica)

Material: cc798424-97c3-4830-92a6-25c942ad01fe
	refractive_index
		<refractive_index, 1.458 dimensionless, None>

	relative_permittivity
		<relative_permittivity, 3.9 dimensionless, None>

	relative_permeability
		<relative_permeability, 0.5450676923076923 dimensionless, None>



# Working with Models

A Model object is used to represent a relationship between different materials property variables. This object can be directly manipulated and stores relavent metadata available as direct attributes.

- All Models are imported as classes at runtime.
- A Model class must be instantiated to be used at runtime.

In [3]:
from propnet.models import *
model = RefractiveIndexfromRelPerm()
print(model.description)
print(model.name)
print()
print(model.equations)


The refractive index gives the factor by which the speed of light is reduced in a medium.

Likewise, modeling the induced magnetic and electric dipoles as linear within a material,
a relative spatial electrical permittivity and relative spatial magnetic permeability
arise from consideration of the total electrical and magnetic fields.

From the Maxwell Relations, the index of refraction is equal to the geometric mean  of the
relative permittivity and the relative permeability.

RefractiveIndexfromRelPerm

['n - sqrt(Ur*Er)']


## Model Class Structure - Basic Overview

The Model class is a generally-defined interface, and subclasses may alter many aspects of its underlying functionality.


Most Model objects will contain equations, symbols, and connections attributes. These define the core functionality of the model:

The equations attribute will contain a list of sympy-parsable expressions. These expressions imply trivial equations such that the expression is equal to zero. 
(ie. x - 4 for corresponding equation x = 4)

In [4]:
print(model.equations)

['n - sqrt(Ur*Er)']


The symbol_mapping attribute maps the symbols (ie. n) used in the equations to Symbol objects (ie. refractive index) used in the Property Network.

In [5]:
print(model.symbol_mapping)

{'Er': 'relative_permeability', 'Ur': 'relative_permittivity', 'n': 'refractive_index'}


The connections attribute shows what outputs can be generated from a set of inputs, following a standard format.

In [6]:
print(model.connections)

[{'inputs': ['Ur', 'Er'], 'outputs': ['n']}, {'inputs': ['Er', 'n'], 'outputs': ['Ur']}, {'inputs': ['Ur', 'n'], 'outputs': ['Er']}]


A Model can be evaluated to generate outputs if given a complete set of inputs.

Given the relative permeability and permittivity, the Refractive Index From Relative Permeability model can correctly calculate the index of refraction.

This is given by 'n' in the dictionary below.

In [7]:
model.evaluate({'Ur': 0.54, 'Er': 3.9})

{'n': <Quantity(1.451206394693739, 'dimensionless')>, 'successful': True}

# Loading Materials Data

Material properties can be loaded in from the Materials Project so they don't need to be defined and added manually.

Accessing Materials Project data requires an API key. You must input your own API key below to run the sample. You can locate your api key by logging into materialsproject.org and visiting the dashboard.

In [16]:
from propnet.ext.matproj import *
my_api_key = None
silica = import_material('mp-546794', api_key=my_api_key)
print(silica)

Material: 55a6cdb6-dd27-4fa3-afc3-b44445e47b67
	structure:	Full Formula (Si2 O4)
Reduced Formula: SiO2
abc   :   5.138209   5.139163   5.138919
angles:  88.540850 120.840203 120.841618
Sites (6)
  #  SP           a         b         c    coordination_no  forces
---  ----  --------  --------  --------  -----------------  --------------------------------------
  0  O     0.334692  0.375014  0.209679                  4  [0.00907491, -0.00223112, 0.00638321]
  1  Si    0.999993  1.00001   0.999974                  4  [0.00399411, 0.00268543, 0.00019724]
  2  O     0.834699  0.959681  0.625017                  4  [0.00353084, 0.00703423, -0.00908806]
  3  O     0.165321  0.790307  0.12502                   4  [-0.00778536, -0.00920807, 0.00757932]
  4  O     0.665314  0.875015  0.040304                  4  [-0.01303024, -0.000511, -0.00656695]
  5  Si    0.49998   0.749969  0.250005                  4  [0.00421573, 0.00223052, 0.00149524]
	lattice_unit_cell:	[[ 4.59584983 -0.71705099 -2.182

# Working with Units

## Symbol Objects

No units need to be specified when defining a Quantity object or plugging values into Models. In the case that no units are supplied, default units are inferred based on those provided in the corresponding Symbol object.

For instance, youngs_modulus is a Symbol with the default units of gigapascal. 
In the example above:

           youngs_modulus = Symbol('youngs_modulus', 200, [])
           
youngs_modulus automatically has the value 200 gigapascals despite no units being provided.

It is possible to provide your own units by working with the <b>unit registry</b> (ureg).<br></br>
The examples below demonstrate this capability using the method: 
            
            ureg.Quantity(value, unit)
            
The property network will automatically convert the units you supply into canonical units behind the scenes for use in models that use the property.

In [11]:
from propnet import ureg
from propnet.core.graph import Graph
from propnet.core.materials import Material
from propnet.core.quantity import Quantity

g = Graph()

mild_steel = Material()
yield_strength = Quantity('yield_stress', ureg.Quantity(53700, 'psi'), [])
mild_steel.add_quantity(yield_strength)
print(mild_steel)

Material: a23091ec-9486-4856-9ea3-a352c153fc6a
	yield_stress
		<yield_stress, 0.3702484666431411 gigapascal, None>



## Models
No units need to be specified when plugging values into a Model. In the case that no units are supplied, default units are inferred based on those provided in the corresponding SymbolType objects.

            model.evaluate(dict)
            
The above method can be used to plug in raw python floats, indicating that default units are implied. 

The above method can also be used with custom units supplied to one or more of the values in the dictionary. 

In the example below we use the property network to calculate the Peierls-Nabarro Stress for a disloaction on the (111) plane in Aluminum (FCC) using custom units of psi for shear modulus.

In [12]:
model = PeierlsStress()
G = ureg.Quantity(3.9*10**6, 'psi')
a = 4.046 / 3**(1/2)  # Angstroms automatically assumed by the model.
b = 4.046 / 2**(1/2)  # Angstroms automatically assumed by the model.
nu = 0.33

tau = model.evaluate({'G': G, 'a': a, 'b': b, 'nu': nu})
print(tau['T_pn'])

0.012711228013154436 gigapascal


## Converting Units
Given you have an output with units, it is straightforward to convert between units using Pint.

In [14]:
# Converting to megapascals.
print(tau['T_pn'].to('megapascal'))

12.711228013154436 megapascal


# Creating Custom Models and Properties

The property network comes with many different models and properties pre-loaded and ready for use.

To add additional properties to the project can be accomplished in two different forms:

- Defining additional .yaml and .py files in the propnet.models and propnet.symbols folders respectively. For these changes to take effect, the property network must be re-loaded.

- Defining additional Model and SymbolType classes at runtime.

Both approaches will be detailed below.

## Defining new model files:
Models require a model_name.yaml and a model_name.py file to be defined in the propnet/models folder.

The syntax for a model.yaml definition is as follows:

<table style="width:100%">
  <tr>
    <th>Field Name</th>
    <th>Field Format</th> 
    <th>Field Description</th>
  </tr>
  <tr>
    <td>title</td>
    <td>string</td> 
    <td>title of the model</td>
  </tr>
  <tr>
    <td>tags</td>
    <td>list of strings</td> 
    <td>any user-specific tags applying to the model</td>
  </tr>
  <tr>
    <td>references</td>
    <td>list of strings</td> 
    <td>any urls for citing the model</td>
  </tr>
  <tr>
    <td>symbol_mapping</td>
    <td>dictionary, string keys, string values</td> 
    <td>mapping from symbols used in the model to SymbolType names used in the property network</td>
  </tr>
  <tr>
    <td>connections</td>
    <td>list of dictionaries with keys: 'inputs', 'outputs' mapping to lists of symbol strings used in the model</td> 
    <td>datastrucutre used to determine the set of all outputs that can be generated from a set of inputs</td>
  </tr>
  <tr>
    <td>equations</td>
    <td>list of strings</td> 
    <td>list of equations that create the model, defined as the value of the expression equals zero</td>
  </tr>
</table>

By default the python file can be a simple class declaration:

            class model_name(AbstractModel):
                pass
                
Methods of the AbstractModel class can be overridden to allow for custom model evaluation() or constraint handling (see below).

## Defining new Symbol files:
Symbols require a symbol.yaml file to be defined in the propnet/symbols folder.
The syntax for a symbol_type.yaml definition is as follows:
<table style="width:100%">
  <tr>
    <th>Field Name</th>
    <th>Field Format</th> 
    <th>Field Description</th>
  </tr>
  <tr>
    <td>name</td>
    <td>string</td> 
    <td>name of the property, must match the file name</td>
  </tr>
  <tr>
    <td>units</td>
    <td>nested list: [1.0, [[unit name string, unit_power], ...]</td> 
    <td>the units used to represent this SymbolType, adjacent entries in the list indicate multiplication.</td>
  </tr>
  <tr>
    <td>display_names</td>
    <td>list of strings</td> 
    <td>any strings useful for representing this SymbolType</td>
  </tr>
  <tr>
    <td>display_symbols</td>
    <td>list of strings</td> 
    <td>any short string symbols useful for representing this SymbolType</td>
  </tr>
  <tr>
    <td>dimension</td>
    <td>list of numbers</td> 
    <td>datastructure to give the length of each dimension in a higher order tensor representation of the property, 1 for a scalar value.</td>
  </tr>
  <tr>
    <td>comment</td>
    <td>string</td> 
    <td>any comments pertinent to the SymbolType</td>
  </tr>
  <tr>
    <td>category</td>
    <td>string</td> 
    <td>whether this is a property, object, or condition</td>
  </tr>
</table>

## Defining Models at Runtime:
A Model can be defined as a new class that extends the AbstractModel class.

See example below for a simple overview. All information is loaded in through the metadata optional argumnent. This defines a model that takes in values (a, b) and returns their product.

Based on symbol_mapping, this model expects variable a to be of type A, variable b to be of type B, and outputs a variable c of type C.

For more advanced behavior, additional methods of AbstractModel could be overridden for custom model evaluation() or constraint handling (see below).

In [15]:
from propnet.core.models import AbstractModel
class Model1 (AbstractModel):
    def __init__(self, symbol_types=None):
        AbstractModel.__init__(self, metadata={
                        'title': 'model1',
                        'tags': [],
                        'references': [],
                        'symbol_mapping': {'a': 'A',
                                           'b': 'B',
                                           'c': 'C'
                                           },
                        'connections': [{
                                         'inputs': ['a', 'b'],
                                         'outputs': ['c']
                                         }],
                        'equations': ['c=a*b'],
                        'description': ''
            },
            symbol_types=symbol_types)

## Defining SymbolTypes at Runtime:
A new SymbolType can be defined as a new instance of the SymbolType class by specifying all the metadata present in a symbol_type.yaml file.

See example below.

In [16]:
from propnet.core.symbols import Symbol
A = Symbol('gibbs_free_energy', ['G'], ['G'], units=[1.0, [['joules',1]]], shape=1)

# A Grand Example

In [19]:
silicon = Material()
silicon.add_quantity(Quantity('temperature', 300, []))
silicon.add_quantity(Quantity('band_gap', 1.12, []))
silicon.add_quantity(Quantity('electrical_conductivity', 4.34E-4, []))

aluminum = Material()
aluminum.add_quantity(Quantity('temperature', 300, []))
aluminum.add_quantity(Quantity('band_gap', 0, []))
aluminum.add_quantity(Quantity('electrical_conductivity', 3.69E7, []))

p1 = Graph(materials=[silicon])
p1.evaluate()

p2 = Graph(materials=[aluminum])
p2.evaluate()

print("Silicon's Graph")
print(silicon)
print()
print("Aluminum's Graph")
print(aluminum)

Silicon's Graph
Material: cecd6280-5cc4-4d0a-8cad-b33604a2faa5
	temperature
		<temperature, 300 kelvin, None>

	band_gap
		<band_gap, 1.12 electron_volt, None>

	electrical_conductivity
		<electrical_conductivity, 0.000434 / meter / ohm, None>

	refractive_index
		<refractive_index, 3.1941119999999996 dimensionless, None>
		<refractive_index, 3.034774286526527 dimensionless, None>

	band_gap_pbe
		<band_gap_pbe, 1.5081602373887242 electron_volt, None>

	is_metallic
		<is_metallic, 0 dimensionless, None>

	band_gap_gw
		<band_gap_gw, 1.1362725450901805 electron_volt, None>


Aluminum's Graph
Material: 3b132658-c573-437b-9f32-4fd33dacee44
	temperature
		<temperature, 300 kelvin, None>

	band_gap
		<band_gap, 0 electron_volt, None>

	electrical_conductivity
		<electrical_conductivity, 36900000.0 / meter / ohm, None>

	band_gap_pbe
		<band_gap_pbe, 0.6772997032640949 electron_volt, None>

	is_metallic
		<is_metallic, 1 dimensionless, None>

	band_gap_gw
		<band_gap_gw, 0.014028056112224449

# Models with Custom Evaluation

In several cases, a model may not be representable as an equation parsable by sympy. In these cases, the plug_in method must be overridden to perform the custom evaluation. 

The is_metallic model is reproduced below, yielding a simple example of custom evaluation.

In [20]:
class IsMetallic(AbstractModel):
    def plug_in(self, symbol_values):
        return {'is_metallic': symbol_values['E_g'] <= 0}

# A Grand Example